In [67]:
#Rename the CSV file to "Heart_Disease_Mortality" bc the name is WAY TOO LONG
#Open in Google Collaboratory, save changes frequently, and we cannot use the file at the same time

In [68]:
import numpy as np
import random as random
import pandas as pd
import json,csv, requests, re
from bs4 import BeautifulSoup

In [69]:
#This the CSV stuff

In [75]:
#import pandas as pd  #already imported delete later
df = pd.read_csv('Heart_Disease_Mortality.csv')
  
#removes the columns
df.drop('Year', inplace=True, axis=1)
df.drop('GeographicLevel', inplace=True, axis=1)
df.drop('DataSource', inplace=True, axis=1)
df.drop('Class', inplace=True, axis=1)
df.drop('Topic', inplace=True, axis=1)
df.drop('Data_Value_Unit', inplace=True, axis=1)
df.drop('Data_Value_Type', inplace=True, axis=1)
df.drop('Data_Value_Footnote_Symbol', inplace=True, axis=1)
df.drop('Data_Value_Footnote', inplace=True, axis=1)
df.drop('StratificationCategory1', inplace=True, axis=1)
df.drop('Stratification1', inplace=True, axis=1)
df.drop('StratificationCategory2', inplace=True, axis=1)
df.drop('Stratification2', inplace=True, axis=1)
df.drop('TopicID', inplace=True, axis=1)
df.drop('LocationID', inplace=True, axis=1)
df.drop('Location 1', inplace=True, axis=1)
df.drop('LocationDesc', inplace=True, axis=1)

#getting rid of the columns with np.nan in the 'Data_Value' column
df = df[df['Data_Value'].notna()]

#sorting the datavalues by 'LocationAbbr'
df.sort_values(by=['LocationAbbr'], ascending=True, inplace = True)

#resets the index of each row to increase from 0 to the number of rows
df.reset_index(inplace = True, drop = True)

#Finding the average number of deaths due to Heart Disease per state/territory per 100,000 people
df = df.groupby(['LocationAbbr']).agg({'Data_Value': 'mean'})

#Rename the Data_Value column to Avg_Num_Deaths
df.rename(columns = {'Data_Value':'Avg_Num_Deaths'}, inplace = True)

#Finding the total number of deaths within in the US states and territories, this can be used to find the percentage that each state makes up
total = 0
for row in avg_deaths['Avg_Num_Deaths']:
    total += row

#Adding a new column to avg_deaths that represents the percentage of heart disease deaths in the country that fall within the state
percentage  = []
for row in df['Avg_Num_Deaths']:
    percentage.append(row/total*100)
df['Percent Total %'] = percentage

#Writing the dataframe to an output CSV file
df.to_csv('heart_disease_output.csv', index=True)

print(df)

              Avg_Num_Deaths  Percent Total %
LocationAbbr                                 
AK                318.396517         1.698614
AL                452.940996         2.416396
AR                447.272996         2.386157
AS                343.533333         1.832716
AZ                272.962357         1.456227
CA                298.068489         1.590166
CO                247.797399         1.321975
CT                255.655882         1.363899
DC                306.178571         1.633432
DE                270.705172         1.444185
FL                286.450326         1.528184
GA                380.160799         2.028120
GU                529.800000         2.826431
HI                300.981707         1.605708
IA                329.146016         1.755962
ID                278.923204         1.488028
IL                343.175380         1.830807
IN                354.644851         1.891995
KS                339.465344         1.811014
KY                431.482062      

In [71]:
#This is the BeautifulSoup Stuff

In [74]:
#Url of the website with the table
r = requests.get("https://en.wikipedia.org/wiki/Obesity_in_the_United_States")

#Create a BeautifulSoup object
soup = BeautifulSoup(r.text,"html.parser")

#Find the table tag
table = soup.find('table',{'class':'wikitable sortable'})

#This is the list where the final data will be appended
output_list = []

#Find the header row and getting the column headers (tr), appending it to the final list
header_row = table.find('tr')
headers = header_row.find_all('th')
header_info = []
for header in headers:
    info = (header.get_text().strip())
    if "[" in info:
        info = info[:-4]
    if "[" in info:
        info = info[:-4]
    header_info.append(info)
output_list.append(header_info)

#Find all row tags (tr) in the table
rows = table.find_all('tr')

#This is the regex match text
regex_replace = re.compile(r'\[.*?\]')

#Iterate over each row and getting the data from each cell (td)
for row in rows[1:]: #skipping the header row which would otherwise be blank
    info = row.find_all('td')
    row_data = []
    for item in info:
        cleaned_data = (item.get_text().strip())
        if "[" in cleaned_data:
            cleaned_data = cleaned_data[:-4]
        if "[" in cleaned_data:
            cleaned_data = cleaned_data[:-4]
        row_data.append(cleaned_data)
    output_list.append(row_data)

#Replace the '-' with n/a to better represent the meaning of '-'
for lists in output_list:
    x+=1
    for i in range(len(lists)):
        if lists[i] == '—':
            lists[i] = "n/a"

print(output_list)
#Writing the output_list to an output_file using csv
#opening an output file for writing named US_Obesity
with open('US_Obesity.csv', 'w', newline='') as output:
    writer = csv.writer(output)
    writer.writerows(output_list)

[['States, district, & territories', 'Obesity rank', 'Obese adults (mid-2000s)', 'Obese adults (2020)', 'Overweight (incl. obese) adults(mid-2000s)', 'Obese children and adolescents(mid-2000s)'], ['Alabama', '5', '30.1%', '36.3%', '65.4%', '16.7%'], ['Alaska', '9', '27.3%', '34.2%', '64.5%', '11.1%'], ['American Samoa', 'n/a', 'n/a', '75%', '95%', '35%'], ['Arizona', '30', '23.3%', '29.5%', '59.5%', '12.2%'], ['Arkansas', '7', '28.1%', '35.0%', '64.7%', '16.4%'], ['California', '48', '23.1%', '25.1%', '59.4%', '13.2%'], ['Colorado', '51', '21.0%', '22.6%', '55.0%', '9.9%'], ['Connecticut', '42', '20.8%', '26.9%', '58.7%', '12.3%'], ['Delaware', '23', '25.9%', '31.8%', '63.9%', '22.8%'], ['District of Columbia', '50', '22.1%', '23.0%', '55.0%', '14.8%'], ['Florida', '35', '23.3%', '28.4%', '60.8%', '14.4%'], ['Georgia', '24', '27.5%', '31.6%', '63.3%', '16.4%'], ['Guam', 'n/a', 'n/a', '28.3%', 'n/a', '22%'], ['Hawaii', '49', '20.7%', '23.8%', '55.3%', '13.3%'], ['Idaho', '32', '24.6%', 

In [73]:
#Inconsistencies to Note
#CSV: Removed all the rows that had np.nan as the data value
#BeautifulSoup: Since this is a wikipedia page, there are footnotes. Those were removed using string methods. 
#BeautifulSoup: Replaced the '-' in the data set with "n/a" to better represent what the absent values represent. Additionally, by not removing any of the rows, there are an equal number of states/territories (57) in both the wikipedia dataset and the csv dataset